In [1]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

In [2]:
# default: Load the model on the available device(s)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct", dtype="auto", device_map="auto"
)

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


In [3]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "../data/test1.jpeg",
            },
            {"type": "text", "text": "이 사진 속에서 위험요소를 찾아주고 개선방안도 알려줘. + "}, 
        ],
    }
]

In [4]:
# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

In [5]:
# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=1024)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
# print(output_text)

for text in output_text:
    print(text)

이 사진에서 주요 위험 요소와 그에 따른 개선 방안은 다음과 같습니다:

1. **전기 케이블**: 흰색 전기 케이블이 놓여 있으며, 이는 안전한 위치에 있지 않습니다. 특히, 케이블이 다른 물건 위에 놓여 있거나, 다른 물건과 접촉하는 경우, 화재의 위험이 있습니다.

   - **개선 방안**: 케이블을 안전하게 관리하고, 다른 물건과 분리하여 놓아야 합니다. 필요하다면, 케이블을 바닥에 고정하거나, 케이블 관을 사용하여 관리할 수 있습니다.

2. **전자제품의 위치**: 전자제품들이 모두 같은 공간에 놓여 있습니다. 이는 화재나 전기 누설의 위험을 증가시킬 수 있습니다.

   - **개선 방안**: 각 전자제품이 따로 있는 공간에 놓여야 합니다. 예를 들어, 미니 오븐은 다른 공간에, 커피 머신은 다른 공간에 놓여야 합니다. 또한, 각 장치가 충분한 공간을 가지고 있어야 하며, 충분한 공기가 통과할 수 있도록 해야 합니다.

3. **물건의 위치**: 물건들이 미끄러지거나 넘어질 위험이 있습니다. 특히, 커피 머신과 커피 컵이 같은 공간에 놓여 있습니다.

   - **개선 방안**: 물건들이 미끄러지거나 넘어질 위험을 줄이기 위해, 물건들이 안정적으로 놓여야 합니다. 필요하다면, 물건들을 안전하게 고정하거나, 물건들이 미끄러지지 않도록 하는 방법을 찾아야 합니다.

4. **화재 위험**: 미니 오븐과 커피 머신이 같은 공간에 놓여 있으며, 이는 화재의 위험을 증가시킬 수 있습니다.

   - **개선 방안**: 각 전자제품이 따로 있는 공간에 놓여야 합니다. 또한, 전자제품들이 충분한 공간을 가지고 있어야 하며, 충분한 공기가 통과할 수 있도록 해야 합니다. 또한, 전자제품들이 서로 가까워지지 않도록 하여 화재의 위험을 줄일 수 있습니다.

이러한 위험 요소를 해결하기 위해서는 전문가의 도움을 받는 것이 좋습니다.
